In [1]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
import random
import json
from collections import Counter
import string
from nltk.stem.porter import *

In [2]:
reviewList = []
with open('reviewSelected100.json', encoding = "ISO-8859-1") as datafile:
    for jsonObj in datafile:
        reviewDict = json.loads(jsonObj)
        reviewList.append(reviewDict)

In [3]:
businessList = []
for review in reviewList:
    businessList.append(review["business_id"])
# random_businessid = random.choice(businessList)
# print(type(random_businessid))
# print("Random business b1 selected: "+random_businessid)

Select a random business 'p6FPcgLymnpk_gAyQuW_Mw', separate review of the selected business and other businesses

In [4]:
reviews = []
reviewsOther = []
for review in reviewList:
    #if (review["business_id"] == random_businessid): # considering all reviews for same business b1
    if (review["business_id"] == 'p6FPcgLymnpk_gAyQuW_Mw'):
        reviews.append(review["text"].lower())
    else:
        reviewsOther.append(review["text"].lower())

Find all ADJ phrases in reviews of the selected business

In [5]:
from nltk.parse import CoreNLPParser
parser = CoreNLPParser(url='http://localhost:9000')

Tokenize sentence

In [6]:
sentences = []
for review in reviews:
    sentences.extend(sent_tokenize(review))

In [7]:
ADJPs = []
for sentence in sentences:
    try:
        structure = list(parser.raw_parse(sentence)) 
        ADJPs.extend(list(structure[0].subtrees(lambda t: t.label() == 'ADJP')))
    except:
        print("error occurs when processing: " + sentence)

error occurs when processing: return factor- 100%


In [8]:
ngramADJP = {}  
for ADJP in ADJPs:
    if len(list(ADJP.subtrees(lambda t: t.label() == 'ADJP'))) == 1:
        words = ADJP.leaves();
        expression = ""
        for i in range(len(words)):
            word = words[i]
            if word in string.punctuation:
                continue
            expression += word.strip().lower()
            if i != len(words) - 1:
                expression += " "
        n = len(expression.split())
        if n > 5:
            continue
        if n == 1:
            continue;
        if(n not in ngramADJP):
            ngramADJP[n] = [expression]
        else:
            ngramADJP[n].append(expression)

In [9]:
from nltk.util import bigrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import word_tokenize
from nltk.lm import Laplace
from nltk.tokenize import wordpunct_tokenize
from scipy.stats import entropy
import math

In [14]:
def trainlm(reviews):
    reviews_str = ''.join(reviews)
    test_corpus = [list(wordpunct_tokenize(reviews_str))]
    train, vocab = padded_everygram_pipeline(5, test_corpus)
    lm = MLE(5)
    lm.fit(train, vocab)
    return lm

In [15]:
def getNgramProbability(words,lm):
    frequency = lm.logscore(words[0])
    for i in range(1,len(words)):
        frequency += lm.logscore(words[i],[words[i-1]])
    return frequency

In [16]:
lm_b = trainlm(reviews)
lm_other = trainlm(reviewsOther)
ADJPprobability = {}
min_n = min(ngramADJP.keys())
max_n = max(ngramADJP.keys())
print("calculating ngram probability in reviews for business b1")
for n in range(min_n, max_n + 1):
    for phrase in ngramADJP[n]:
        words = phrase.split()
        ADJPprobability[phrase] = getNgramProbability(words,lm_b)
ADJPprobabilityInOtherBusiness = {}
print("calculating ngram probability in reviews for other business")
for n in range(min_n, max_n + 1):
    for phrase in ngramADJP[n]:
        words = phrase.split()
        ADJPprobabilityInOtherBusiness[phrase] = getNgramProbability(words,lm_other)
relativeEntropy = {}
for ADJP in ADJPprobability.keys():
    probabilityInB = math.exp(ADJPprobability[ADJP])
    probabilityInOther = math.exp(ADJPprobabilityInOtherBusiness[ADJP])
    relativeEntropy[ADJP] = entropy([probabilityInB, 1-probabilityInB], qk=[probabilityInOther, 1-probabilityInOther])
sortedADJP = sorted(relativeEntropy.items(),key=lambda d:d[1],reverse = True)
sortedADJP

calculating ngram probability in reviews for business b1
calculating ngram probability in reviews for other business


[('turbo charged', inf),
 ('bodega ever', inf),
 ('insanely delicious', inf),
 ('plentiful too', inf),
 ('leaderless american', inf),
 ('kinda soggy', inf),
 ('pippin hot', inf),
 ('pure awesomeness ', inf),
 ('so legit', inf),
 ('delicious spicy', inf),
 ('surprisingly organized', inf),
 ('disgusting orange', inf),
 ('perfect by themselves', inf),
 ('amazing late night', inf),
 ('extraordinary vey impressive', inf),
 ('huuuge and delicious', inf),
 ('fastest and biggest', inf),
 ('kind of scary', inf),
 ('not really mexican', inf),
 ('multicultural and diverse', inf),
 ('convenient in convenience', inf),
 ('new to tempe', inf),
 ('a little finer', inf),
 ('drab and dingy', inf),
 ('definition college town convenient', inf),
 ('uninviting from the outside', inf),
 ('open late', 6.382396550574292e-05),
 ('so good', 3.6894924866140707e-05),
 ('super chill', 3.2430805486873855e-05),
 ('super hungry', 2.8473982386990288e-05),
 ('really good', 2.203713500010146e-05),
 ('very dry', 2.1217938